In [1]:
pip install numpy music21 tensorflow sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [2]:
#DataFlair Automatic Music Generation Project
#load all the libraries
from music21 import *
import glob
from tqdm import tqdm
import numpy as np
import random
from tensorflow.keras.layers import LSTM,Dense,Input,Dropout
from tensorflow.keras.models import Sequential,Model,load_model
from sklearn.model_selection import train_test_split

In [3]:
import glob
from tqdm import tqdm
from music21 import converter, instrument, note, chord

def read_files(file):
    notes = []
    notes_to_parse = None
    
    # Parse the midi file
    midi = converter.parse(file)
    
    # Separate all instruments from the file
    instrmt = instrument.partitionByInstrument(midi)

    for part in instrmt.parts:
        # Fetch data only of Piano instrument
        if 'Piano' in str(part):
            notes_to_parse = part.recurse()

            # Iterate over all the parts of sub stream elements
            # Check if element's type is Note or chord
            # If it is chord, split them into notes
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
    
    # Return the list of notes
    return notes

# Modify the file_path variable to your directory path where MIDI files are stored
file_path = r"C:\Users\tjase\OneDrive\Desktop\AUTOMATIC MUSIC PROJECT\MIDI Files\bach"
all_files = glob.glob(file_path + '/*.mid', recursive=True)

# Reading each midi file
notes_list = [read_files(i) for i in tqdm(all_files, position=0, leave=True)]


100%|██████████| 3/3 [00:01<00:00,  1.84it/s]


In [4]:
#unique notes
notess = sum(notes_list, [])
unique_notes = list(set(notess))
print("Unique Notes:", len(unique_notes))

#notes with their frequency
freq = dict(map(lambda x: (x, notess.count(x)), unique_notes))

#get the threshold frequency
for i in range(30, 100, 20):
    print(i, ":", len(list(filter(lambda x: x[1] >= i, freq.items()))))

Unique Notes: 114
30 : 36
50 : 28
70 : 18
90 : 17


In [5]:
# Filter notes greater than threshold i.e., 50
freq_notes = dict(filter(lambda x: x[1] >= 50, freq.items()))

# Create new notes using the frequent notes
new_notes = [[i for i in j if i in freq_notes] for j in notes_list]


In [6]:
#dictionary having key as note index and value as note
ind2note=dict(enumerate(freq_notes))

#dictionary having key as note and value as note index
note2ind=dict(map(reversed,ind2note.items()))

In [7]:
#timestep
timesteps=50

#store values of input and output
x=[] ; y=[]

for i in new_notes:
  for j in range(0,len(i)-timesteps):
    #input will be the current index + timestep
    #output will be the next index after timestep
    inp=i[j:j+timesteps] ; out=i[j+timesteps]

    #append the index value of respective notes 
    x.append(list(map(lambda x:note2ind[x],inp)))
    y.append(note2ind[out])

x_new=np.array(x) 
y_new=np.array(y)

In [8]:
#reshape input and output for the model
x_new = np.reshape(x_new,(len(x_new),timesteps,1))
y_new = np.reshape(y_new,(-1,1))

#split the input and value into training and testing sets
#80% for training and 20% for testing sets
x_train,x_test,y_train,y_test = train_test_split(x_new,y_new,test_size=0.2,random_state=42)

In [9]:
#create the model
model = Sequential()
#create two stacked LSTM layer with the latent dimension of 256
model.add(LSTM(256,return_sequences=True,input_shape=(x_new.shape[1],x_new.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))

#fully connected layer for the output with softmax activation
model.add(Dense(len(note2ind),activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 256)           264192    
                                                                 
 dropout (Dropout)           (None, 50, 256)           0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dense_1 (Dense)             (None, 28)                7196      
                                                                 
Total params: 862492 (3.29 MB)
Trainable params: 862492 

In [10]:

#compile the model using Adam optimizer
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

#train the model on training sets and validate on testing sets
model.fit(
    x_train,y_train,
    batch_size=128,epochs=80, 
    validation_data=(x_test,y_test))

Epoch 1/80
20/20 [==============================] - 12s 407ms/step - loss: 3.2763 - accuracy: 0.0439 - val_loss: 3.2428 - val_accuracy: 0.0788
Epoch 2/80
20/20 [==============================] - 10s 525ms/step - loss: 3.2358 - accuracy: 0.0656 - val_loss: 3.2094 - val_accuracy: 0.0772
Epoch 3/80
20/20 [==============================] - 10s 525ms/step - loss: 3.2111 - accuracy: 0.0781 - val_loss: 3.2174 - val_accuracy: 0.0595
Epoch 4/80
20/20 [==============================] - 10s 511ms/step - loss: 3.2044 - accuracy: 0.0857 - val_loss: 3.1807 - val_accuracy: 0.0820
Epoch 5/80
20/20 [==============================] - 11s 529ms/step - loss: 3.1812 - accuracy: 0.0821 - val_loss: 3.1819 - val_accuracy: 0.0772
Epoch 6/80
20/20 [==============================] - 10s 495ms/step - loss: 3.1691 - accuracy: 0.0861 - val_loss: 3.1658 - val_accuracy: 0.0884
Epoch 7/80
20/20 [==============================] - 9s 455ms/step - loss: 3.1700 - accuracy: 0.0849 - val_loss: 3.1745 - val_accuracy: 0.0820


In [11]:
#save the model for predictions
model.save(r"C:\Users\tjase\OneDrive\Desktop\AUTOMATIC MUSIC PROJECT\new prediction")

INFO:tensorflow:Assets written to: C:\Users\tjase\OneDrive\Desktop\AUTOMATIC MUSIC PROJECT\new prediction\assets


INFO:tensorflow:Assets written to: C:\Users\tjase\OneDrive\Desktop\AUTOMATIC MUSIC PROJECT\new prediction\assets


In [20]:
# load the model
model = load_model(r"C:\Users\tjase\OneDrive\Desktop\AUTOMATIC MUSIC PROJECT\new prediction")
# generate random index
index = np.random.randint(0, len(x_test) - 1)
# get the data of generated index from x_test
music_pattern = x_test[index]
out_pred = []  # it will store predicted notes

# iterate till 200 note is generated
for i in range(200):
    # reshape the music pattern
    music_pattern = music_pattern.reshape(1, len(music_pattern), 1)

    # get the maximum probability value from the predicted output
    pred_index = np.argmax(model.predict(music_pattern))
    # get the note using predicted index and
    # append to the output prediction list
    out_pred.append(ind2note[pred_index])
    music_pattern = np.append(music_pattern, pred_index)

    # update the music pattern with one timestep ahead
    music_pattern = music_pattern[1:]

1/1 [==============================] - 0s 34ms/step


In [25]:
from music21 import note, chord, instrument, stream

output_notes = []

for offset, pattern in enumerate(out_pred):
    # if pattern is a chord instance
    if ('.' in pattern) or pattern.isdigit():
        # split notes from the chord
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            i_curr_note = int(current_note)
            # cast the current note to Note object and
            # append the current note
            new_note = note.Note(i_curr_note)
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)

        # cast the current note to Chord object
        # offset will be 1 step ahead from the previous note
        # as it will prevent notes to stack up
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)

    else:
        # cast the pattern to Note object, apply the offset, and
        # append the note
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

# Save the midi file with the name 'hafiz.mid'
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi',fp='new2.mid')


'new2.mid'

In [24]:
# Save the midi file with the name 'hafiz.mid'
midi_stream = stream.Stream(output_notes)
midi_stream.write(fp='new2.mid')


WindowsPath('c:/Users/tjase/OneDrive/Desktop/new2.mid')